# CS 1656 – Introduction to Data Science 

## Instructors: Alexandros Labrinidis, Xiaowei Jia
## Teaching Assistants: Evangelos Karageorgos, Xiaoting Li, Zi Han Ding
### Additional credits: Tahereh Arabghalizi, Zuha Agha, Anatoli Shein, Phuong Pham
## Recitation : SQL via Data API
---
In this recitation, you will execute SQL queries on real data by connecting to the open data portal of [Western Pennsylavnia Regional Data Center](https://www.wprdc.org/) and requesting data via API calls.  

In [229]:
import json
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

We will be using Allegheny County Restaurant/Food Facility Inspection Violation Dataset found here https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations. This dataset contains violation data from actual routine inspections by one of health department staff's members for the last two years. It should be fun to find out inspection results for places where we eat in Pittsburgh! =)

In [230]:
wprdc_api_endpoint = "https://data.wprdc.org/api/3/action/datastore_search_sql"

# id for database table
resource_id = "1a1329e2-418c-4bd3-af2c-cc334e7559af"

# Get the date from 270 days ago)
# end_date = datetime.now()
# start_date = end_date - timedelta(days=270)

# Get two date endpoints
start_date = date(2021, 9, 1)
end_date = date(2022, 6, 1)

# Convert to a string the format the the data center accepts (yyyy-mm-dd)
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")

# SQL query we'll use in API call to request data
query = """
SELECT *
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}'""".format(resource_id, start_str, end_str, "Pittsburgh")

# Make WPRDC API Call
response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

# Parse response JSON into python dictionary
response_data = json.loads(response.text)

# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(response_data['result']['records'])

# Print the number of rows
print(df.shape[0], "rows total")
print(df.columns)
df.head()


/Users/skasko/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


10430 rows total
Index(['_id', '_full_text', 'encounter', 'id', 'placard_st', 'facility_name',
       'bus_st_date', 'description', 'description_new', 'num', 'street',
       'city', 'state', 'zip', 'inspect_dt', 'start_time', 'end_time',
       'municipal', 'rating', 'low', 'medium', 'high', 'url'],
      dtype='object')


,_id,_full_text,encounter,id,placard_st,facility_name,bus_st_date,description,description_new,num,...,zip,inspect_dt,start_time,end_time,municipal,rating,low,medium,high,url
0,42154084,'-01':22 '-10':8 '-21':9 '-27':23 '/reports/rw...,202201270025,44110,1,Marshall's #237,1988-10-21,Chain Packaged Food Only,Administrative,1900,...,15220,2022-01-27,15:10:00,15:40:00,Scott,V,None,None,None,http://appsrv.alleghenycounty.us/reports/rwser...
1,42154111,'-04':27 '-06':12 '-07':28 '-104':36 '-17':13 ...,202204070011,44262,1,W PA School For Blind Children Rooms,1984-06-17,Traditional Boarding Home,Garbage and Refuse,201,...,15213,2022-04-07,10:50:00,12:00:00,Pittsburgh-104,V,T,None,None,http://appsrv.alleghenycounty.us/reports/rwser...
2,42154112,'-06':12 '-10':27 '-104':36 '-12':28 '-17':13 ...,202110120016,44262,1,W PA School For Blind Children Rooms,1984-06-17,Traditional Boarding Home,Cleaning and Sanitization,201,...,15213,2021-10-12,10:45:00,12:00:00,Pittsburgh-104,V,T,F,F,http://appsrv.alleghenycounty.us/reports/rwser...
3,42154125,'-06':12 '-10':25 '-104':34 '-12':26 '-17':13 ...,202110120016,44262,1,W PA School For Blind Children Rooms,1984-06-17,Traditional Boarding Home,Plumbing,201,...,15213,2021-10-12,10:45:00,12:00:00,Pittsburgh-104,V,T,F,F,http://appsrv.alleghenycounty.us/reports/rwser...
4,42154126,'-06':12 '-10':30 '-104':39 '-12':31 '-17':13 ...,202110120016,44262,1,W PA School For Blind Children Rooms,1984-06-17,Traditional Boarding Home,"Contamination Prevention - Food, Utensils and ...",201,...,15213,2021-10-12,10:45:00,12:00:00,Pittsburgh-104,V,T,F,F,http://appsrv.alleghenycounty.us/reports/rwser...


Details of useful dataset attributes are below. ((Taken from https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations/resource/1a1329e2-418c-4bd3-af2c-cc334e7559af)

* __facility_name__: the name of the facility
* __description__: Facility category
* __description_new__: The name of the potential violation
* __inspect_dt__: Date/time of the inspection
* __rating__: The result of the inspection ('V' for violation, other for non-violation)
* The health risk of a potential violation
 * __low__: low risk
 * __medium__: medium risk
 * __high__: high risk
* The address of the facility
 * __city__: The city
 * __state__: The state
 * __street__: The street
 * __num__: The street number
 * __zip__: The zip code

## Queries

__Q1) Find all unique decription categories of violation in Pittsburgh restaurants over the time span (violation description[violation]).__

In [356]:
query = """
SELECT DISTINCT "description_new" as violation
FROM "{}"
WHERE "inspect_dt" 
BETWEEN '{}' and '{}' 
AND "city" = '{}' 

""".format(resource_id, start_str, end_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/Users/skasko/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,violation
0,Administrative
1,Certified Food Protection Manager
2,Cleaning and Sanitization
3,Cold Holding Temperatures
4,Consumer Advisory
5,"Contamination Prevention - Food, Utensils and ..."
6,Cooking Temperatures
7,Cooling Food
8,Cross-Contamination Prevention
9,Date Marking of Food


__Q2) Find restaurants in Pittsburgh with no violations in at least one decription category (facility name[facility], number of violations[count]). NOTE: a facility has a violation if the inspection rating has the value 'V'.__

In [37]:
query = """
SELECT "facility_name" as facility, COUNT("description_new") as count
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' AND "rating" <> '{}'
GROUP BY "facility_name" """.format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/Users/skasko/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,facility,count
0,Bethlehem Haven of Pittsburgh Rooms,1
1,Children’s Museum of Pittsburgh,1
2,Fort Pitt Inn,1
3,Fort Pitt Lodge #1,1
4,Global Food Pittsburgh,1
5,Google Pittsburgh,5
6,Harvie Farms Pittsburgh,1
7,Manor Care Health Services Pittsburgh,1
8,Pitt Dining Cathedral Cafe Food Court,1
9,Pitt Dining Litchfield Tower (A) / Market Cent...,1


# Tasks

__Tasks 1 to 4 must be implemented in Task.py.__

__For all tasks, we want the results in Pittsburgh, over the specified time span, and all queries are about violations (rating is 'V').__

__As the API returns the results as a list of dictionaries, the order of the query columns is irrelevant.__


__T1) Find the top 20 facilities that start with 'Pitt' and have the highest counts of violations _(facility name[facility], number of violations[count])_.__

In [412]:
query = """

            SELECT "facility_name" as facility, COUNT("description_new") as count
            FROM "{}"
            WHERE "inspect_dt" BETWEEN '{}'
            AND '{}'
            AND "city" = '{}' 
            AND "rating" = 'V'
            AND "facility_name" LIKE 'Pitt%'
            GROUP BY facility
            ORDER BY count DESC
            LIMIT 20
            

""".format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)
print(response.text)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df




{"help": "https://data.wprdc.org/api/3/action/help_show?name=datastore_search_sql", "success": true, "result": {"sql": "\n\n            SELECT \"facility_name\" as facility, COUNT(\"description_new\") as count\n            FROM \"1a1329e2-418c-4bd3-af2c-cc334e7559af\"\n            WHERE \"inspect_dt\" BETWEEN '2021-09-01'\n            AND '2022-06-01'\n            AND \"city\" = 'Pittsburgh' \n            AND \"rating\" = 'V'\n            AND \"facility_name\" LIKE 'Pitt%'\n            GROUP BY facility\n            ORDER BY count DESC\n            LIMIT 20\n            \n\n", "records": [{"facility": "Pitt Eats William Pitt Union / Schenley Cafe & Central Catering", "count": 7}, {"facility": "Pitt Eats Litchfield Tower (B) / The Eatery @ The Towers", "count": 7}, {"facility": "Pitt Eats Cathedral of Learning / Cathedral Cafe Food Court", "count": 4}, {"facility": "Pitt Eats Commissary / Posvar Kitchen", "count": 4}, {"facility": "Pittsburgh Sandwich Society @ Strange Roots", "count": 

/Users/skasko/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,facility,count
0,Pitt Eats William Pitt Union / Schenley Cafe &...,7
1,Pitt Eats Litchfield Tower (B) / The Eatery @ ...,7
2,Pitt Eats Cathedral of Learning / Cathedral Ca...,4
3,Pitt Eats Commissary / Posvar Kitchen,4
4,Pittsburgh Sandwich Society @ Strange Roots,3
5,Pitt Eats Parran Hall / Public Health Coffee,3
6,Pitt Eats Sutherland Hall / The Perch,3
7,Pittsburgh Popcorn Co,3
8,Pitt Cathedral of Learning / Saxbys,2
9,Pitt Eats Mervis Hall / Bottom Line Bistro,2


__T2) Find the top 18 restaurants with the maximum number of violations _(facility name[facility], number of violations[count])_. Include all results in case of a tie (For example, if the 18th top restaurant has 10 violations, incude all other restaurants with 10 violations). HINT: You will need an extra query to get the tie-breaker value.__

In [451]:
query = """

    SELECT "facility_name" AS restaurant, COUNT("description_new") AS count
    FROM "{}"
    WHERE "inspect_dt" BETWEEN '{}' AND '{}'
    AND "city" = '{}'
    AND "rating" = 'V'
    GROUP BY "facility_name"
    ORDER BY count DESC
    LIMIT 18, 1


""".format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)
print(response.text)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

{"help": "https://data.wprdc.org/api/3/action/help_show?name=datastore_search_sql", "error": {"query": ["(psycopg2.errors.SyntaxError) LIMIT #,# syntax is not supported\nLINE 10:     LIMIT 18, 1\n             ^\nHINT:  Use separate LIMIT and OFFSET clauses.\n\n[SQL: SELECT * FROM (\n\n    SELECT \"facility_name\" AS restaurant, COUNT(\"description_new\") AS count\n    FROM \"1a1329e2-418c-4bd3-af2c-cc334e7559af\"\n    WHERE \"inspect_dt\" BETWEEN '2021-09-01' AND '2022-06-01'\n    AND \"city\" = 'Pittsburgh'\n    AND \"rating\" = 'V'\n    GROUP BY \"facility_name\"\n    ORDER BY count DESC\n    LIMIT 18, 1\n\n\n) AS blah LIMIT 32001 ;]\n(Background on this error at: https://sqlalche.me/e/14/f405)"], "info": {"statement": ["SELECT * FROM (\n\n    SELECT \"facility_name\" AS restaurant, COUNT(\"description_new\") AS count\n    FROM \"1a1329e2-418c-4bd3-af2c-cc334e7559af\"\n    WHERE \"inspect_dt\" BETWEEN '2021-09-01' AND '2022-06-01'\n    AND \"city\" = 'Pittsburgh'\n    AND \"rating\" 

/Users/skasko/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


KeyError: 'result'

__T3) Find the facilities that start with 'Pitt' and have violations over the time span _(violation description[violation], number of facilities[count], facility names[facilities])_. The _facilities_ field must be a concatenation of all facility names, in alphabetical order, seperated by a comma and a space (', ').__

In [486]:
query = """


            SELECT DISTINCT "description_new" as violation, COUNT("description_new") as count, STRING_AGG("facility_name", ', ') as facility
            FROM "{}"
            WHERE "inspect_dt" BETWEEN '{}'
            AND '{}'
            AND "city" = '{}' 
            AND "rating" = 'V'
            AND "facility_name" LIKE 'Pitt%'
            GROUP BY violation
            ORDER BY violation ASC           

""".format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)
print(response.text)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df


/Users/skasko/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"help": "https://data.wprdc.org/api/3/action/help_show?name=datastore_search_sql", "success": true, "result": {"sql": "\n\n\n            SELECT DISTINCT \"description_new\" as violation, COUNT(\"description_new\") as count, STRING_AGG(\"facility_name\", ', ') as facility\n            FROM \"1a1329e2-418c-4bd3-af2c-cc334e7559af\"\n            WHERE \"inspect_dt\" BETWEEN '2021-09-01'\n            AND '2022-06-01'\n            AND \"city\" = 'Pittsburgh' \n            AND \"rating\" = 'V'\n            AND \"facility_name\" LIKE 'Pitt%'\n            GROUP BY violation\n            ORDER BY violation ASC           \n\n", "records": [{"violation": "Administrative", "count": 2, "facility": "Pittsburgh News, Pittsburgh Playhouse Lobby Bar"}, {"violation": "Cleaning and Sanitization", "count": 6, "facility": "Pitt Eats William Pitt Union / Schenley Cafe & Central Catering, Pittsburgh Popcorn Co, Pitt Cathedral of Learning / Saxbys, Pittsburgh Playhouse Lobby Bar, Pittsburgh Popcorn, Pittsburg

,violation,count,facility
0,Administrative,2,"Pittsburgh News, Pittsburgh Playhouse Lobby Bar"
1,Cleaning and Sanitization,6,Pitt Eats William Pitt Union / Schenley Cafe &...
2,Cold Holding Temperatures,2,Pitt Eats Cathedral of Learning / Cathedral Ca...
3,"Contamination Prevention - Food, Utensils and ...",3,"Pitt Eats Commissary / Posvar Kitchen, Pitt Ea..."
4,Cross-Contamination Prevention,1,Pittsburgh Sandwich Society @ Strange Roots
5,"Fabrication, Design, Installation and Maintenance",17,"Pitt Cathedral of Learning / Saxbys, Pitt Eats..."
6,Facilities to Maintain Temperature,4,Pitt Eats William Pitt Union / Schenley Cafe &...
7,Floors,1,Pitt Eats Litchfield Tower (B) / The Eatery @ ...
8,General Premises,2,Pitt Eats Litchfield Tower (B) / The Eatery @ ...
9,Handwashing Facilities,5,"Pitt Eats Sutherland Hall / The Perch, Pitt Hi..."


Now lets look at all facilities that contain word 'Pitt'.

__T4) Find the category descriptions and their high, medium, low risk ratings for all violations at all facilities that have word 'Pitt' in their name. Note that results that contain word 'Pitt' as part of another word (e.g. 'Pittsburgh') should not be included _(facility name[facility], violation description[violation], high[high], medium[medium], low[low])_. HINT: consider _all_ edge cases for identifying 'Pitt' as a seperate word.__

In [ ]:
query = """

    SELECT "facility_name" AS restaurant, COUNT("description_new") AS count
    FROM "{}"
    WHERE "inspect_dt" BETWEEN '{}' AND '{}'
    AND "city" = '{}'
    AND "rating" = 'V'
    GROUP BY "facility_name"
    ORDER BY count DESC
    LIMIT 18, 1


""".format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)
print(response.text)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df